In [ ]:
# Copyright 2023 Nils Knieling
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 💬 kölschGPT

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-%23F9AB00.svg?logo=googlecolab&logoColor=white)](https://colab.research.google.com/github/Cyclenerd/google-cloud-gcp-openai-api/blob/cologne/koelschGPT.ipynb)
[![Open in Vertex AI Workbench](https://img.shields.io/badge/Open%20in%20Vertex%20AI%20Workbench-%234285F4.svg?logo=googlecloud&logoColor=white)](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/Cyclenerd/google-cloud-gcp-openai-api/cologne/koelschGPT.ipynb)
[![View on GitHub](https://img.shields.io/badge/View%20on%20GitHub-181717.svg?logo=github&logoColor=white)](https://github.com/Cyclenerd/google-cloud-gcp-openai-api/blob/cologne/koelschGPT.ipynb)

This notebook describes how to load documents from Google Drive. Currently, only Google Docs and Google Sheets are supported. Based on these documents and information, a chatbot (like ChatGPT or Bard) optimized for questions about cologne is created and installed in the Google Cloud as a serverless cloud run service.

> By default, [**Google Cloud does not use Customer Data to train its foundation models**](https://cloud.google.com/vertex-ai/docs/generative-ai/data-governance#foundation_model_development) as part of Google Cloud's AI/ML Privacy Commitment. More details about how Google processes data can also be found in [Google's Customer Data Processing Addendum (CDPA)](https://cloud.google.com/terms/data-processing-addendum).

## Setup


### Runtime

>⚠️ You may receive a warning to "Restart Runtime" after the packages are installed. Don't worry, the subsequent cells will help you restart the runtime.

In [ ]:
#@markdown #### Install dependencies

#@markdown * [Google authentication](https://pypi.org/project/google-auth/)
#@markdown * [Google API client](https://pypi.org/project/google-api-python-client/)
#@markdown * [Google Vertex AI API client](https://pypi.org/project/google-cloud-aiplatform/)
#@markdown * [LangChain](https://pypi.org/project/langchain/)
#@markdown * [Facebook AI similarity search](https://pypi.org/project/faiss-cpu/)

!pip install google-auth==2.22.0
!pip install google-auth-oauthlib==1.0.0
!pip install google-auth-httplib2==0.1.0
!pip install google-api-python-client==2.97.0
!pip install google-cloud-storage==2.10.0
!pip install google-cloud-aiplatform==1.31.1

!pip install langchain==0.0.276
!pip install unstructured==0.10.8
!pip install faiss-cpu==1.7.4

print("☑️ Done")

In [ ]:
#@markdown #### Restart

# Automatically restart kernel after installs so that your environment
# can access the new packages.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Configuration

Folder URL `https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5`, the folder ID is `1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5`

Regions: <https://cloud.google.com/vertex-ai/docs/general/locations#vertex-ai-regions>

Key `openai_key` is used for authentication the chat frondend against the API backend application. It is not a real OpenAI API key.

In [ ]:
# @markdown ✏️ Replace the placeholder text below:

# Please fill in these values.
project_id = "your-google-cloud-project"  # @param {type:"string"}
region = "europe-west1"  # @param {type:"string"}
vector_bucket = "your-gcs-bucket-for-faiss-index"  # @param {type:"string"}
artifact_repository = "docker-koelschgpt"  # @param {type:"string"}
folder_id = "1iUrKVsEfVNW0CuzuF5JZw22aNfU0KjPD"  # @param {type:"string"}
openai_key = "sk-do-ming-stadt-am-rhing" # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert vector_bucket, "⚠️ Please provide a Google Cloud storage bucket for the vector store"
assert artifact_repository, "⚠️ Please provide a Google Cloud Artifact repository name"
assert folder_id, "⚠️ Please provide a valid folder ID"
assert openai_key, "⚠️ Please provide a valid key for authentication"

# Configure gcloud.
!gcloud config set project "{project_id}"
!gcloud config set storage/parallel_composite_upload_enabled "True"

print(f"\n☁️ Google Cloud project ID: {project_id}")
print(f"📁 Google Drive folder ID: {folder_id}")
print("☑️ Done")

### Authenticate

In [ ]:
#@markdown #### (Colab only!) Authenticate your Google Cloud Account

# Authenticate gcloud.
from google.colab import auth
auth.authenticate_user()

print("☑️ OK")

In [ ]:
#@markdown ####  Check authenticated user
current_user = !gcloud auth list \
  --filter="status:ACTIVE" \
  --format="value(account)" \
  --quiet

current_user = current_user[0]
print(f"Current user: {current_user}")

### Import

In [ ]:
#@markdown #### Import Python libraries

import sys
import time
from typing import List
from pydantic import BaseModel
from IPython.display import display, Markdown

print(f"🐍 Python: {sys.version}")

# Vertex AI
# https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm
from google.cloud import aiplatform

aiplatform.init(
    project=project_id,
    location=region,
)

print(f"☁️ Vertex AI SDK version: {aiplatform.__version__}")

# Langchain
import langchain
from langchain.llms import VertexAI
# https://python.langchain.com/docs/integrations/document_loaders/google_drive
from langchain.document_loaders import GoogleDriveLoader
# https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm
from langchain.embeddings import VertexAIEmbeddings
# Facebook AI Similarity Search (FAISS), local Vector Store
# https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/
from langchain.vectorstores import FAISS

print(f"🦜️ LangChain version: {langchain.__version__}")

### APIs

In [ ]:
#@markdown #### Enable Google Cloud APIs
#@markdown > Only necessary if the APIs are not yet activated in the project!

# Enable APIs
my_google_apis = [
    "storage.googleapis.com",
    "aiplatform.googleapis.com",
    "drive.googleapis.com",
    "run.googleapis.com",
    "artifactregistry.googleapis.com",
    "cloudbuild.googleapis.com",
    "containeranalysis.googleapis.com",
    "containerscanning.googleapis.com",
]

for api in my_google_apis :
  print(f"Enable API: {api}")
  !gcloud services enable "{api}" \
    --project="{project_id}" \
    --quiet

print("☑️ OK")

### Storage

In [ ]:
#@markdown #### Create storage bucket for verctor store
#@markdown > Only necessary if the bucket does not already exist!

!gcloud storage buckets create 'gs://{vector_bucket}' \
  --location='{region}' \
  --uniform-bucket-level-access \
  --quiet

print("☑️ Done")

In [ ]:
#@markdown #### Add default compute service account as viewer to bucket

project_number = !gcloud projects list \
  --filter="{project_id}" \
  --format="value(PROJECT_NUMBER)" \
  --quiet

project_number = project_number[0]

print(f"Project number: {project_number}")

!gcloud storage buckets add-iam-policy-binding "gs://{vector_bucket}" \
  --member="serviceAccount:{project_number}-compute@developer.gserviceaccount.com" \
  --role="roles/storage.objectViewer" \
  --quiet

print("☑️ Done")

### Registry

In [ ]:
#@markdown #### Create Artifact Registry repositoriy for Docker cointainer images
#@markdown > Only necessary if the repositoriy does not already exist in the project and region!

!gcloud artifacts repositories create "{artifact_repository}" \
  --repository-format="docker"\
  --description="Docker contrainer registry" \
  --location="{region}" \
  --project="{project_id}" \
  --quiet

print("☑️ Done")

## Documents

### Load

In [ ]:
#@markdown #### Load documents from Google Drive folder

loader = GoogleDriveLoader(
    folder_id=folder_id,
    file_types=["document", "sheet"],
    recursive=False
)
docs = loader.load()

for doc in docs:
  title = doc.metadata['title']
  source = doc.metadata['source']

  display(Markdown(f"📄 [{title}]({source})"))

### Transform

In [ ]:
#@markdown #### Split Google Workspace documents into chunks

# Split long text descriptions into smaller chunks that can fit into
# the API request size limit, as expected by the LLM providers.

text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)

chunked = text_splitter.split_documents(docs)

print("Preview:")
print(chunked[0].page_content, "\n")
print(chunked[1].page_content)

print("☑️ Done")

### Embed

In [ ]:
#@markdown #### Generate vector embeddings
#@markdown > ⏳ This code snippet may run for a few minutes!

# Embedding
embeddings = VertexAIEmbeddings(
    project=project_id,
    max_retries=12,
    request_parallelism=5,
)
# Embed your texts
db = FAISS.from_documents(chunked, embeddings)

# Store index
faiss_index = "faiss_index" # filename
db.save_local(faiss_index)

# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retriever

print("\n☑️ Done")

### Test

In [ ]:
# @markdown Enter search query in a simple English text.
db_query = "When is the cathedral open?"  # @param {type:"string"}

# Quick input validations.
assert db_query, "⚠️ Please input a valid input search text"

retriever_docs = retriever.get_relevant_documents(db_query)

print("\n\n".join([retriever_doc.page_content[:400] for retriever_doc in retriever_docs[:2]]))

### Copy

In [ ]:
# @markdown #### Copy vector store to Google Cloud storage bucket
# @markdown This bucket and verctor store is later used by the chat bot (AI).

!gcloud storage cp -R {faiss_index}/* 'gs://{vector_bucket}/' \
  --quiet

> 💡 Info: If you stored new documents in the Google Drive folder or changed the documents you always have to regenerate (load, transform, embed) and save/copy the new vector store. To be sure that the new index is used I also recommend a re-deployment of the Cloud Run API backend service.

## Deploy

### Backend

In [ ]:
#@markdown #### Build Docker cointainer for API backend

backend_git = "https://github.com/Cyclenerd/google-cloud-gcp-openai-api.git"  # @param {type:"string"}
backend_git_rev = "cologne"  # @param {type:"string"}
assert backend_git, "⚠️ Please provide a Git repo"
assert backend_git_rev, "⚠️ Please provide a Git revision"

!gcloud builds submit "{backend_git}" \
    --git-source-revision="{backend_git_rev}" \
    --tag="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/vertex:cologne" \
    --timeout="1h" \
    --region="{region}" \
    --default-buckets-behavior="regional-user-owned-bucket" \
    --quiet

In [ ]:
#@markdown #### Deploy Cloud Run service with API backend

!gcloud run deploy "cologne-api" \
    --image="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/vertex:cologne" \
    --description="Vertex AI ob Kölsch" \
    --region="{region}" \
    --set-env-vars="OPENAI_API_KEY={openai_key},GOOGLE_CLOUD_LOCATION={region},VECTOR_BUCKET={vector_bucket},VECTOR_DIR=/vector,FAISS_INDEX=/vector" \
    --max-instances=4 \
    --allow-unauthenticated \
    --quiet

### Frontend

Git repos:

*   [Chatbot UI](https://github.com/mckaywrigley/chatbot-ui): `https://github.com/mckaywrigley/chatbot-ui.git`
*   [Chatbot UI fork](https://github.com/Cyclenerd/chatbot-ui) with less OpenAI branding: `https://github.com/Cyclenerd/chatbot-ui.git`



In [ ]:
#@markdown #### Build Docker cointainer for chat frontend


frontend_git = "https://github.com/Cyclenerd/chatbot-ui.git"  # @param {type:"string"}
frontend_git_rev = "main"  # @param {type:"string"}

assert frontend_git, "⚠️ Please provide a Git repo"
assert frontend_git_rev, "⚠️ Please provide a Git revision"

!gcloud builds submit "{frontend_git}" \
    --git-source-revision="{frontend_git_rev}" \
    --tag="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/chatbot-ui:main" \
    --timeout="1h" \
    --region="{region}" \
    --default-buckets-behavior="regional-user-owned-bucket" \
    --quiet

In [ ]:
#@markdown #### Deploy Cloud Run service with chat frontend

# @markdown ✏️ Replace the Cloud Run service URL from the backend below:

backend_url = "https://cologne-api-yjbgpbvhaq-ew.a.run.app"  # @param {type:"string"}
assert backend_url, "⚠️ Please provide a Cloud Run backend URL"

!gcloud run deploy "koelschgpt" \
    --image="{region}-docker.pkg.dev/{project_id}/{artifact_repository}/chatbot-ui:main" \
    --description="kölschGPT" \
    --region="{region}" \
    --set-env-vars="OPENAI_API_KEY={openai_key},OPENAI_API_HOST={backend_url}" \
    --max-instances=2 \
    --allow-unauthenticated \
    --quiet

## 🗑️ Clean up

If you don't need the infrastructure anymore, you can delete it with the following snippets.

In [ ]:
#@markdown ### Delete Cloud Run service with chat frontend

!gcloud run services delete "koelschgpt" \
  --region="{region}" \
  --project="{project_id}" \
  --quiet

In [ ]:
#@markdown ### Delete Cloud Run service with API backend

!gcloud run services delete "cologne-api" \
  --region="{region}" \
  --project="{project_id}" \
  --quiet

In [ ]:
#@markdown ### Delete bucket for vector index

!gcloud storage rm -r 'gs://{vector_bucket}' \
  --project="{project_id}" \
  --quiet

print("☑️ Done")

In [ ]:
#@markdown ### Delete Artifact Registry repositoriy
!gcloud artifacts repositories delete "{artifact_repository}" \
  --location="{region}" \
  --project="{project_id}" \
  --quiet